In [1]:
cd ~/bench

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [2]:
import pandas as pd
import polars as pl
from pathlib import Path

In [4]:
raw_data_path = Path("data/source_tables/raw")
yadl_data_path = Path("data/source_tables/yadl")
od_data_path = Path("data/source_tables/open_data_us")


# Company employees

In [4]:
df = pl.read_parquet(Path(raw_data_path, "company-employees-yadl.parquet"))

In [5]:
df.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "company-employees-depleted-yadl.parquet")
)
df.select(pl.col("name"), pl.col("target")).write_parquet(
    Path(od_data_path, "company-employees-depleted_name-open_data.parquet")
)

# US Accidents

In [24]:
df = pl.read_parquet(Path(raw_data_path, "us-accidents-yadl.parquet"))
df.drop("col_to_embed").write_parquet(Path(od_data_path, "us_accidents-open_data.parquet"))

In [23]:
df.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "us_accidents-depleted-yadl.parquet")
)
df.select(pl.col("County"), pl.col("target")).write_parquet(
    Path(od_data_path, "us_accidents-depleted_County-open_data.parquet")
)

# US Elections

In [25]:
df = pl.read_parquet(Path(yadl_data_path, "us_elections-yadl.parquet"))
df.drop("col_to_embed").write_parquet(Path(od_data_path, "us_elections-open_data.parquet"))

In [13]:
df.select(pl.col("target"), pl.col("party"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "us_elections-depleted-yadl.parquet")
)
df.select(pl.col("county_name"), pl.col("party"), pl.col("target")).write_parquet(
    Path(od_data_path, "us_elections-depleted_county_name-open_data.parquet")
)

### Prepare versions of US elections filtered by party

In [ ]:
df_filtered = df.filter(pl.col("party") == "DEMOCRAT")
df_filtered.select(
    pl.col("state"),
    pl.col("county_name"),
    pl.col("party"),
    pl.col("target"),
    pl.col("col_to_embed"),
).write_parquet(Path(raw_data_path, "us-elections-dems.parquet"))

df_filtered.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "us-elections-dems-depleted-yadl.parquet")
)
df_filtered.select(pl.col("county_name"), pl.col("target")).write_parquet(
    Path(od_data_path, "us-elections-dems-depleted-open_data.parquet")
)

# Movies revenue

In [26]:
df = pl.read_parquet(Path(raw_data_path, "movies-yadl.parquet"))
df.drop("col_to_embed").write_parquet(Path(od_data_path, "movies-open_data.parquet"))


In [15]:
df.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "movies-depleted-yadl.parquet")
)
df.select(pl.col("title"), pl.col("target")).write_parquet(
    Path(od_data_path, "movies-depleted_title-open_data.parquet")
)

# Movies votes

In [27]:
df = pl.read_parquet(Path(raw_data_path, "movies-vote-yadl.parquet"))
df.drop("col_to_embed").write_parquet(Path(od_data_path, "movies_vote-open_data.parquet"))

In [17]:
df.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(yadl_data_path, "movies_vote-depleted-yadl.parquet")
)
df.select(pl.col("title"), pl.col("target")).write_parquet(
    Path(od_data_path, "movies_vote-depleted_title-open_data.parquet")
)

# Housing Prices

In [28]:
df = pl.read_parquet(Path(yadl_data_path, "housing_prices-yadl.parquet")).drop("col_to_embed")
df.drop("col_to_embed").write_parquet(Path(od_data_path, "housing_prices-open_data.parquet"))

In [19]:
df.select(pl.col("target"), pl.col("County")).write_parquet(
    Path(od_data_path, "housing_prices-depleted_County-open_data.parquet")
)
df.select(pl.col("target"), pl.col("Metro")).write_parquet(
    Path(od_data_path, "housing_prices-depleted_Metro-open_data.parquet")
)


# Metam Housing Prices

In [ ]:
df = pl.read_csv(Path(raw_data_path, "housing.csv"), infer_schema_length=10000)
df = df.rename({"URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)": "URL", "ZIP OR POSTAL CODE": "ZIP"})
df = df.with_columns(pl.col("PRICE").log10().alias("target")).drop("PRICE")

In [ ]:
df.select(pl.col("target"), pl.col("ZIP")).write_parquet(
    Path(od_data_path, "housing-depleted_ZIP-open_data.parquet")
)
df.select(pl.col("target"), pl.col("CITY")).write_parquet(
    Path(od_data_path, "housing-depleted_CITY-open_data.parquet")
)
df.write_parquet(
    Path(od_data_path, "housing-open_data.parquet")
)

# Metam Schools

In [7]:
df = pl.read_csv(Path(od_data_path, "schools.csv"), infer_schema_length=10000)

In [8]:
df = df.with_columns(pl.col("class").cast(pl.Int8).alias("target")).drop("class")
df = df.rename({"DBN": "col_to_embed"})

In [9]:
df.select(pl.col("target"), pl.col("col_to_embed")).write_parquet(
    Path(od_data_path, "schools-depleted-open_data.parquet")
)
df.write_parquet(
    Path(od_data_path, "schools-open_data.parquet")
)